# Pages App

- class-bassed views
- templates
- test templates

## Initial Set up

- make a new directory for our code called pages and navigate into it
- create a new virtual environment called . venv and activate it
- install Django 

- create a new Django project called django_project2

- create anew app called pages

```bash
mkdir django_project2

cd django_project2

python3.8 -m venv .venv

source .venv/bin/activate

pip install django

django-admin startproject django_project2 .

python manage.py migrate

python manage.py runserver
```

- add APP to project:

```python
# django_project2/settings.py 
INSTALLED_APPS = [
"django.contrib.admin", 
"django.contrib.auth", 
"django.contrib.contenttypes", 
"django.contrib.sessions", 
"django.contrib.messages", 
"django.contrib.staticfiles", 
"pages", # new 
] |
```

## Templates

Every web framework needs a convenient way to generate HTML files, and
in Django, the approach is to use templates: individual HTML files that can
be linked together and include basic logic.

Recall that our “Hello, World” site had the phrase hardcoded into a
views.py file in the previous chapter. 
That technically works but does not scale well. 
A better approach is to link a view to a template, thereby separating the information contained in each. 


- create a single project-level templates directory and place all templates within it - By tweaking our django_project/settings.py ﬁle, we can tell Django to look in
this directory for templates. 

create a directory called templates:

```bash
mkdir templates
```

file structure (`tree -I '__pycache__'`):

```
├── db.sqlite3
├── django_project2
│   ├── asgi.py
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
├── manage.py
├── pages
│   ├── admin.py
│   ├── apps.py
│   ├── __init__.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── tests.py
│   ├── urls.py
│   └── views.py
└── templates
    └── home.html
```

## Class-Based Views 
Early versions of Django only shipped with function-based views, but
developers soon found themselves repeating the same patterns over and over.

1. Write a view that lists all objects in a model. 
2. Write a view that displays only one detailed item from a model. 
And so on. 

Generic function-based views were introduced to abstract these patterns and streamline the development of common patterns. 

To help with code reusability, Django added class-based views and generic
class-based views while still retaining function-based views. 


There are three different ways to write a view in Django:

1. function-based
2. class-based
3. generic class-based

- Function-based views are simpler to understand because they 3
mimic the HTTP request/response cycle 
- Class-based views are a little harder to understand because their inheritance structure means you have to dive into the code to see everything happening; 
- And generic class-based views are the hardest yet to understand. 

An entire website, Classy Class-Based Views, is dedicated to helping developers
decipher them.
So why bother with generic class-based views? Once you have used them for
a while, they become elegant and efficient ways to write code. You can often
modify a single method on one to do custom behavior rather than rewriting
everything from scratch, which makes it easier to understand someone else’s
code. This does, however, come at the cost of complexity 

When a generic class-based view is not enough, modify it to suit your needs. 
And if that still isn’t enough, revert to a function-based or class-based view. 

In our specific use case, we will use TemplateView (one of the generic views) to display our template.
Replace the default text in the pages/views.py file with the following:
Code

```python
# pages/views.py
from django.views.generic import TemplateView

class HomePageView(TemplateView):
template_name = "home.html" 
```


### About Page

1. add about.html

```
├── django_project2
├── manage.py
├── pages
└── templates
    ├── about.html
    └── home.html
```

2. create About view
3. add view to pages/urls

## Extending Templates

The real power of templates is that they can be extended. 
If you think about most websites, the same content appears on every page:
- header, 
- footer, 
- etc.

- Let’s create a base.html file containing a header with links
to our two pages. 
- We could name this file anything, but using base.html

```
|── templates
    ├── about.html
    ├── base.html
    └── home.html
```

Django has a minimal templating language for adding links and basic logic
in our templates. 
You can see the complete list of built-in template tags
in the official docs. 

Template tags take the form of {% something %} where the “something” is the template tag itself. 
You can even create custom
template tags (out of scope)

To add URL links in our project, we can use the built-in URL template tag
‘which takes the URL pattern name as an argument. 
Remember how we added optional URL names to our two routes in pages/urls.py? 
The url tag uses these names to create links for us automatically.

The URL route for our homepage is called home. To configure a link to it, we
use the following syntax: {% url 'home' %}.

```html
#base.html
<header>
  <a href="{% url 'home' %}">Home</a>
  <a href="{% url 'about' %}">About</a>
</header>

{% block content %} {% endblock content %}
```

At the bottom, we've added a block tag called content. 

- Let’s update our home.html and about.html files to extend the base.html template. 
- That means we can reuse the same code from one template in another. 
- The Django templating language comes with an extends method that we can use for this.

```html
#Home
{% extends "base.html" %}


{% block content %}
<h1>Home</h1>
{% endblock content %}
```


```html
{% extends "base.html" %}

{% block content %}
<h1>About</h1>
{% endblock content %}

```


## Test

- It’s important to add automated tests and run them whenever a codebase
changes. 
- Tests require a small amount of upfront time to write but more than
pay off later on. 
- In the words of Django co-creator Jacob Kaplan-Moss, “Code
without tests is broken as designed.”

The Python standard library contains a built-in testing framework called
unittest that uses TestCase instances and a long list of assert methods
to check for and report failures. 

Django’s testing framework provides several extensions on top of Python’s 
unittest.TestCase base class.

These include a test client for making dummy Web browser requests,
several Django-specific additional assertions, some test case classes:

1. SimpleTestCase, 
2. TestCase, 
3. TransactionTestCase

Generally speaking, SimpleTestCase is used when a database is
unnecessary while TestCase is used when you want to test the database.
TransactionTestCase is helpful to directly test database transactions





If you look within our pages app, Django already provided a tests.py
file we can use. 
Since no database is involved in our project, we will import
SimpleTestCase at the top of the file. 

For our first tests, we’ll check that the two URLs for our website, the homepage and about page, both return HTTP status codes of 200, the standard response for a successful HTTP request.

```python
# pages/tests.py
from django.test import SimpleTestCase

# Create your tests here.
class HomePageTests(SimpleTestCase):
    def test_url_exists(self):
        response = self.client.get('/')
        self.assertEqual(response.status_code, 200)

class AboutPageTest(SimpleTestCase):
    def test_url_exists(self):
        response = self.client.get('/about/')
        self.assertEqual(response.status_code, 200)
```

What else can we test? 
At the moment we are testing the actual URL route for
each page: / for the homepage and /about for the about page. 
But remember that we also added a URL name for each in the pages/urls.py file. 
We should check that the URL name works as well.

To do that we can use the very handy Django utility function **reverse**. 
Instead of going to a URL path first, it looks for the URL name. 
In general, it is a bad idea to hardcode URLs, especially in templates. 
By using reverse we can avoid this. 

For now, we want to test the URL names for our two  pages. 
Import reverse at the top of the file add then add a new unit test for each below.

```python
from django.test import SimpleTestCase
from django.urls import reverse

# Create your tests here.
class HomePageTests(SimpleTestCase):
    def test_url_exists(self):
        response = self.client.get('/')
        self.assertEqual(response.status_code, 200)

    def test_url_available_by_name(self):
        response = self.client.get(reverse('home'))
        self.assertEqual(response.status_code, 200)

class AboutPageTest(SimpleTestCase):
    def test_url_exists(self):
        response = self.client.get('/about/')
        self.assertEqual(response.status_code, 200)

    def test_url_available_by_name(self):
        response = self.client.get(reverse('about'))
        self.assertEqual(response.status_code, 200)
```

Let’s make sure that the correct templates 
-home.html and about.html-
are used on each page and that they display the expected content of

"<hl1>Homepage</h1>" and "<h1>About page</h1>" 

respectively.
We can use *assertTemplateUsed* and *assertContains* to achieve this.

### Request and Response object

- All views always receive a first argument that contains a request object.
- it has attributes like:
1. request.method
2. request.GET
3. request.POST
4. request.COOKIES
etc.

- The response object is a file-like object (a I/O stream) and has the usual methods.
- response = HttpResponse() # returns response
- render also returns response object
- All views must always return 
a response object.




### Function based versus class based

```python
## function based view
def homePageView(request):
    print(request)
```

```python
class HomePageView(TemplateView):
    template_name = 'home.html'

    def get_context_data(self, **kwargs):
        print(self.request)

```

### Respond see code (in django_project2/pages/view.py HomepageView vs. homepageView)

## Custom template tags

```
├── django_project2
├── manage.py
├── pages
│   ├── admin.py
│   ├── apps.py
│   ├── __init__.py
│   ├── models.py
│   ├── templatetags
│   │   ├── init.py
│   │   └── my_tags.py
│   ├── tests.py
│   ├── urls.py
│   └── views.py
├── README.md
└── templates
    ├── about.html
    ├── base.html
    └── home.html
    .....
```

- Template tags are functions.
- Custom template tags must be defined in a directory named templatetags inside an app directory.
- The directory must have an empty __init__.py file.
- The app must be in the INSTALLED_APPS settings constant.
- We can have as many files as we want with any name we want.
- The file with the template tag definition must be loaded in the template, using the load built-in tag.
- The template tag function must be registered in the library.
- It can be done with the simple_tag decorator.

### simple tag

templates/home2.html 
```python

{% load my_tags %}

<h1>{% hello_world %}</h1>
```
django_project2/pages/templatetags/my_tags.py:

```python
from django import template

register = template.Library()

@register.simple_tag
def hello_world():
    return 'Hello World'
```




- Multiple template tag modules, custom and built-in, can be loaded.
- Custom tags may also accept arguments.
- The inclusion_tag decorator is a shortcut for a template renderer.

### inclusion_tag

The inclusion_tag is a feature of Django's templating system that lets you define a template tag that renders a specific template. 

### Step 1: Define the `inclusion_tag` to accept an argument

Update `footer_tags.py`:

```python
# myapp/templatetags/footer_tags.py

from django import template
from datetime import datetime

register = template.Library()

@register.inclusion_tag('footer.html')
def render_footer(company_name="My Company"):
    current_year = datetime.now().year
    return {'current_year': current_year, 'company_name': company_name}
```

### Step 2: Create the template to be included

Ensure `footer.html` is set to handle the company name:

```html
<!-- templates/footer.html -->

<footer>
    Copyright &copy; {{ current_year }} {{ company_name }}.
</footer>
```

### Step 3: Use the `inclusion_tag` with an argument in another template

Load the templatetag and use `render_footer` with a specified company name:

```html
{% load footer_tags %}

... rest of your template ...

{% render_footer "AwesomeCo" %}
```

Now, when this template is rendered, the footer will display:

```
Copyright © [current_year] AwesomeCo.
```

By providing default values in the inclusion tag definition (like "My Company" in our example), you ensure that the tag remains flexible. It can be used with or without specifying the company name. If no name is provided, it'll default to "My Company".

## Creating a Custom Split Filter in Django

### 1. Writing the Filter Function

Navigate to your app's directory and create a `templatetags` directory if it doesn't already exist. Within the `templatetags` directory, create a Python file, e.g., `custom_filters.py`.

Inside `custom_filters.py`, start by importing the necessary modules and then write your custom filter function:

```python
from django import template

register = template.Library()

@register.filter(name='custom_split')
def custom_split(value, arg):
    """
    Splits the value by a given string (arg).
    """
    return value.split(arg)
```

This `custom_split` function takes in a value (which should be a string) and an argument (the delimiter by which to split the string) and returns a list of substrings.

### 2. Using the Filter in Templates

Before using the custom filter, ensure that your app is added to the `INSTALLED_APPS` list in your project's `settings.py`.

In any template where you'd like to use the filter:

1. Load the custom filters at the top of your template:

```django
{% load custom_filters %}
```

2. Use the filter in the template:

```django
{{ "This,is,a,test"|custom_split:"," }}
```

The output will be a list: `['This', 'is', 'a', 'test']`

### 3. Advanced Usage (Optional)

You can expand upon this basic filter by adding more features. For instance, you can handle cases where the delimiter is not provided (default to a space) or limit the number of splits.

### 4. Conclusion

With the custom `custom_split` filter in place, you have more flexibility in splitting strings in your templates. Custom filters in Django provide a powerful way to introduce reusable transformations and manipulations that can be applied directly in the template layer.

#### for loop tag and if tag

In Django templates, you can also combine the `{% for %}` loop with conditional `{% if %}` statements to iterate over lists and display content conditionally. Here's how you can do that:

### Django Template Example:

Let's assume you have a list of users and you want to display only those who are active:

Context:
```python
users = [
    {'name': 'Alice', 'active': True},
    {'name': 'Bob', 'active': False},
    {'name': 'Charlie', 'active': True},
    {'name': 'David', 'active': False}
]
```

Template:
```html
<ul>
{% for user in users %}
    {% if user.active %}
        <li>{{ user.name }}</li>
    {% endif %}
{% endfor %}
</ul>
```

In this example:
- The template iterates over each `user` in the `users` list.
- For each `user`, it checks the `active` attribute using the `{% if %}` tag.
- If `user.active` is `True`, it displays the user's name inside a list item.

The rendered HTML will only show the names of active users:

```html
<ul>
    <li>Alice</li>
    <li>Charlie</li>
</ul>
```

This combination of `{% for %}` and `{% if %}` tags in Django templates allows for powerful yet readable conditional rendering based on lists or other iterable data structures.

## Using a Mixin in a Django View

Mixins are a way to reuse code across multiple class-based views. They can be particularly useful when certain patterns of behavior are needed across multiple views but don't necessarily warrant a full-fledged parent class. In this example, we will create a mixin that ensures a user is active (i.e., not deactivated) before they can access the view.

### 1. Defining the Mixin

First, let's define a mixin that checks if the user accessing the view is active. If the user is not active, they'll be redirected to a custom error page.

Create a file named `mixins.py` in your app's directory and add the following code:

```python
from django.http import HttpResponseForbidden

class ActiveUserRequiredMixin(object):
    """
    Mixin to ensure the user is active.
    """
    
    def dispatch(self, request, *args, **kwargs):
        if not request.user.is_active:
            # This can be replaced with a redirect or a custom error page
            return HttpResponseForbidden("You're not an active user.")
        return super(ActiveUserRequiredMixin, self).dispatch(request, *args, **kwargs)
```

### 2. Applying the Mixin to a View

Now that we have our mixin ready, let's apply it to a view. For this example, we'll use a basic `TemplateView`.

In your `views.py`:

```python
from django.views.generic import TemplateView
from .mixins import ActiveUserRequiredMixin

class DashboardView(ActiveUserRequiredMixin, TemplateView):
    template_name = 'dashboard.html'
```

Note the order of inheritance: the mixin should come before the view base class. This is because the `dispatch` method in the mixin should be the first one to be invoked.

### 3. Configuring the URL

Ensure that the view is reachable by configuring its URL in your `urls.py`.

```python
from django.urls import path
from .views import DashboardView

urlpatterns = [
    path('dashboard/', DashboardView.as_view(), name='dashboard'),
]
```

### 4. Testing the Mixin

1. Run your Django server.
2. Navigate to the `/dashboard/` URL.
3. If you're logged in as an inactive user, you should see the "You're not an active user." message.
4. If you're logged in as an active user or not logged in at all, you should be able to access the dashboard view (or be redirected to the login page if you're using Django's authentication views and the user is not authenticated).

### 5. Conclusion

Mixins allow you to modularize and reuse specific pieces of functionality across different views. The `ActiveUserRequiredMixin` is just one simple example. You can build more complex mixins that handle different types of checks, add context data, or even modify the behavior of the view based on specific conditions.